In [ ]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

# 리스트 URL. 50개씩 곡을 뽑아준다.
loop_url = 'https://www.melon.com/new/index.htm#params%5BareaFlg%5D=I&params%5BorderBy%5D=&po=pageObj&startIndex='
#urlInd = 436600 # 최댓값
urlIndRange = [300,50] #몇 개의 곡을 돌릴지 정한다. 50의 배수 단위로, [최대(436600), 최소(0)]


driver = webdriver.Chrome()
korGenre = ('발라드', '댄스', '랩/힙합', 'R&B/Soul', '인디음악', '록/메탈', '트로트', '포크/블루스', '성인가요/트로트', '클래식', '일렉트로니카', '재즈')
debugMsg = ['페이지 로드 실패', '제목 로드 실패', '가수 로드 실패', '유효하지 않은 장르', '가사 로드 실패', '연도 불명']

lyrics2=[] # 결과 배열 생성
title2=[]
singer2=[]
genre2=[]
date2=[]
songid2=[]

def melon_collector(url):
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    songInfo = soup.find_all('a', attrs={'class': 'btn button_icons type03 song_info'}) # 리스트의 곡을 센다.

    songid=[]
    for i in range(len(songInfo)):
        try:
            songid.append(re.sub(r'[^0-9]', '', songInfo[i]["href"])) # 곡 id정보 추출
        except:
            continue


    idElement = 0

    while idElement < len(songid):

        try:
            debugIndex = 0
            driver.get("https://www.melon.com/song/detail.htm?songId=" + songid[idElement])
            time.sleep(2)
            debugIndex += 1
            title=driver.find_element(by=By.CLASS_NAME, value="song_name").text # 제목 가져오기
            debugIndex += 1
            singer=driver.find_element(by=By.CLASS_NAME, value="artist_name").text # 가수 가져오기
            debugIndex += 1
            genre = driver.find_element(By.XPATH, "//dt[text()='장르']/following-sibling::dd").text.split(',')[0] #장르 가져오기
            if genre not in korGenre: raise Exception() # 장르 검증
            debugIndex += 1
            lyrics=driver.find_element(by=By.CLASS_NAME, value="lyric").text #가사 가져오기
            debugIndex += 1
            date = driver.find_element(By.XPATH, "//dt[text()='발매일']/following-sibling::dd").text[:4]  # 발매 연도 가져오기
            if len(date) < 4: raise Exception() # 연도 검증


        except: # 정보가 없는경우
            print(f"{songid[idElement]} 검증 에러: {debugMsg[debugIndex]}")
            songid.pop(idElement)
            continue

        # 결과 배열에 값 할당
        lyrics2.append(lyrics.replace("\n"," ")) # 줄바꿈 제거
        title2.append(title)
        singer2.append(singer)
        date2.append(date)
        genre2.append(genre)
        songid2.append(songid[idElement])

        idElement += 1

urlInd_tmp = urlIndRange[0]
while (urlInd_tmp > 49 + urlIndRange[1]): #곡이 더 이상 없거나 인덱스가 정해진 최소값에 도달하면 멈춘다.
    start = time.time()
    print(f"({urlIndRange[0]}/{urlIndRange[0] - urlInd_tmp+1 + urlIndRange[1]}) 크롤링 진행 중 . . .")
    melon_collector(loop_url + str(urlInd_tmp-49))
    end = time.time()
    urlInd_tmp -= 50
    print(f"({urlIndRange[0]}/{urlIndRange[0] - urlInd_tmp + urlIndRange[1]}) {end - start:.5f}  sec 소요.\n\n")

print("크롤링 완료")
df=pd.DataFrame({"Id":songid2, "Title":title2, "Singer":singer2, "Lyrics":lyrics2, "Date":date2 , "Genre":genre2})
df.to_csv(f'CrawlingSet{urlIndRange[0]}_{urlInd_tmp+1}.csv', index = False, encoding='utf-8-sig') # 저장하기

(300/51) 크롤링 진행 중 . . .
36891259 검증 에러: 유효하지 않은 장르
36891296 검증 에러: 유효하지 않은 장르
36891057 검증 에러: 가사 로드 실패
(300/100) 143.06300  sec 소요.


(300/101) 크롤링 진행 중 . . .
36892340 검증 에러: 가사 로드 실패
36893036 검증 에러: 가사 로드 실패
(300/150) 135.92491  sec 소요.


(300/151) 크롤링 진행 중 . . .
(300/200) 127.69151  sec 소요.


(300/201) 크롤링 진행 중 . . .
(300/250) 130.75099  sec 소요.


(300/251) 크롤링 진행 중 . . .
36911179 검증 에러: 가사 로드 실패
(300/300) 123.84747  sec 소요.


크롤링 완료
